In [1]:
# Home assignment from Wirecard.
# received 16.01 
# Notebook for Task2

# Task 2

Calculate for each customer three parameters:
* x: Number of transactions done by customer minus 1, it is 0 in case the customer made only one transaction
* tx: Duration in weeks between customer’s last transaction and first transaction, it is 0 in case the customer made only one transaction
* T: Duration in weeks between end of calibration period (01.10.1997) and customer’s first transaction

Save these parameters Customer ID, x, tx, T for each customer in a new csv file called summary_customers.csv. This file is the input for the next step.

In [2]:
#Import packages
import pandas as pd
import numpy as np
import scipy as sp
from scipy.special import gamma
from scipy.special import gammaln as lnG
from scipy import optimize


import datetime as dt

# Task 2

In [3]:
# read the .csv file 
df = pd.read_csv("cal_period_transactions.csv",index_col='Unnamed: 0')
# inspect the data
print(df.shape)
df.head()

(4814, 2)


,ID,date
0,1,19970101
1,1,19970118
2,1,19970802
3,2,19970101
4,2,19970113


## Parameters calculation 

### Convert to a proper datetime format

In [4]:
df['datetime'] = pd.to_datetime(df['date'],format='%Y%m%d')

### x: number of transactions done by customer

In [5]:
df['x'] = df.groupby('ID')['ID'].transform(lambda s: s.count() - 1)
df.head()

,ID,date,datetime,x
0,1,19970101,1997-01-01,2
1,1,19970118,1997-01-18,2
2,1,19970802,1997-08-02,2
3,2,19970101,1997-01-01,1
4,2,19970113,1997-01-13,1


### tx: duration in weeks between customer's last and first transaction

In [6]:
# create temp df with first and last date columns
df_temp = df.datetime.groupby(df['ID']).agg(['first','last'])

In [7]:
# find the difference between the transactions in days
df_temp['tx_days'] = (df_temp['last'] - df_temp['first'])
# convert it to weeks
df_temp['tx'] = df_temp['tx_days']/np.timedelta64(1,'W')
# round to 2 decimals as in the example
df_temp['tx'] = df_temp['tx'].round(2)

In [8]:
df_temp.head()

,first,last,tx_days,tx
ID,,,,
1,1997-01-01,1997-08-02,213 days,30.43
2,1997-01-01,1997-01-13,12 days,1.71
3,1997-01-01,1997-01-01,0 days,0.00
4,1997-01-01,1997-01-01,0 days,0.00
5,1997-01-01,1997-01-01,0 days,0.00


### T: Duration in weeks between end of calibration period and the first customer's transaction 

In [9]:
# from the previous task
split_date_e = 19971001 # Oct 1997
# convert the date for the end of the training period to datetime format and compute the difference
df_temp['T_days'] = dt.datetime.strptime(str(split_date_e), '%Y%m%d') - df_temp['first'] - np.timedelta64(1,'D')# -1 to take the last day into account
# convert it to weeks
df_temp['T'] = df_temp['T_days'] / np.timedelta64(1,'W')
# round to 2 decimals as in the example
df_temp['T'] = df_temp['T'].round(2)

In [10]:
df_temp.head(5)

,first,last,tx_days,tx,T_days,T
ID,,,,,,
1,1997-01-01,1997-08-02,213 days,30.43,272 days,38.86
2,1997-01-01,1997-01-13,12 days,1.71,272 days,38.86
3,1997-01-01,1997-01-01,0 days,0.00,272 days,38.86
4,1997-01-01,1997-01-01,0 days,0.00,272 days,38.86
5,1997-01-01,1997-01-01,0 days,0.00,272 days,38.86


### Merge into the initial DF

In [11]:
df = pd.merge(df,df_temp.drop(columns=['first','last','tx_days','T_days']),on=['ID'])

In [12]:
df.head()

,ID,date,datetime,x,tx,T
0,1,19970101,1997-01-01,2,30.43,38.86
1,1,19970118,1997-01-18,2,30.43,38.86
2,1,19970802,1997-08-02,2,30.43,38.86
3,2,19970101,1997-01-01,1,1.71,38.86
4,2,19970113,1997-01-13,1,1.71,38.86


### Prepare the output df

In [13]:
# only unique customers should be stored for the future fitting
df_out = df.drop_duplicates(subset=['ID'])

In [14]:
# Drop coolumns not needed for the next task
df_out = df_out.reset_index().drop(columns=['date','datetime','index'])
# rename ID column
df_out.rename(columns={'ID': 'Customer ID'},inplace=True)
# Save into .csv file
df_out.to_csv('summary_customers.csv')

In [15]:
df_out.head()

,Customer ID,x,tx,T
0,1,2,30.43,38.86
1,2,1,1.71,38.86
2,3,0,0.00,38.86
3,4,0,0.00,38.86
4,5,0,0.00,38.86
